In [1]:
# Загрузить библиотеки
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [2]:
# Сделать значение цветового канала первым
#K.set_image_data_format("channels_first")

# Задать информацию об изображении
channels = 1
height = 28
width = 28

In [3]:
# Загрузить данные и цель из набора данных MNIST рукописных цифр  
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# # Реформировать тренировочные данные об изображениях в признаки
data_train = data_train.reshape(data_train.shape[0], height, width, channels)

# # Реформировать тестовые данные об изображениях в признаки
data_test = data_test.reshape(data_test.shape[0], height, width, channels)

# Прошкалировать пиксельную интенсивность в диапазон между 0 и 1
features_train = data_train / 255
features_test = data_test / 255

# Преобразовать цель в кодировку с одним активным состоянием
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

In [4]:
# Инициализировать нейронную сеть
network = Sequential()

# Добавить сверточный слой с 64 фильтрами, окном 5x5 и 
# активационной функций ReLU
network.add(Conv2D(filters=64,
                   kernel_size=(5, 5),
                   input_shape=(width, height, channels),
                   activation='relu'))

# Добавить максимально редуцирующий слой с окном 2x2
network.add(MaxPooling2D(pool_size=(2, 2)))

# Добавить отсеивающий слой
network.add(Dropout(0.3))

# Добавить слой для сглаживания входа
network.add(Flatten())

# Добавить полносвязный слой из 128 блоков с
# активационной функций ReLU
network.add(Dense(128, activation="relu"))

# Добавить отсеивающий слой
network.add(Dropout(0.5))

# Добавить полносвязный слой c
# активационной функцией softmax
network.add(Dense(number_of_classes, activation="softmax"))

In [5]:
from Loss_module import keras_loss

In [6]:
callback = keras_loss.LossCallback()

In [8]:
# Скомплировать нейронную сеть
network.compile(
    loss="categorical_crossentropy", # Перекрестная энтропия
    optimizer="rmsprop",  # Распространение СКО
    metrics=["accuracy"]) # Точностный показатель результативности

# Натренировать нейронную сеть
history = network.fit(
    features_train,  # Признаки
    target_train,    # Цель
    epochs=7,        # Количество эпох
    verbose=1,       # Печатать описание после каждой эпохи
    batch_size=1000, # Количество наблюдений на пакет
    validation_data=(features_test, target_test), # Данные для оценивания
    callbacks=[callback]) 

   epoch  test_loss  training_loss
0    1.0   0.151234       0.466225
1    2.0   0.086041       0.178805
2    3.0   0.057564       0.116581
3    4.0   0.048068       0.090750
4    5.0   0.040579       0.073365
5    6.0   0.039911       0.063345
6    7.0   0.035607       0.057636
